In [1]:
import random
import networkx  
import pgmpy.models as pgmm  
import pgmpy.factors.discrete as pgmf              
import pgmpy.inference as pgmi
import random
import numpy

mina = '☠'
casilla_sin_pulsar = '□'

In [2]:
#Entrada de datos del buscaminas
filas = 5
columnas = 5
minas = 10

In [3]:
# Ahora, debemos de crear el tablero del juego, el cual dependerá del numero de filas, columnas y minas
def tablero_buscaminas(filas, columnas, minas):
    # Antes de nada, se debe de comprobar que el número de minas existente en el tablero sea menor o igual al número de casillas
    if(minas > filas * columnas):
        raise Exception ("El número de minas debe ser menor o igual a" + filas*columnas)
        
    # A continuación se pasa a construir el tablero
    tableroBuscaminas = [[0 for x in range(filas)] for y in range(columnas)]
    
    # Vamos a pintar las minas. Recorremos el número de minas que queremos pintar y en el tablero mostramos "X" que indica
    # que en esa casilla existe una mina.
    for i in range(minas):
        tableroBuscaminas[random.randint(0,filas-1)][random.randint(0,columnas-1)] = mina
        
    # Ahora pasamos a poner los índices de las casillas colindantes con minas. El número que aparece en la casilla va en un
    # rango de 1 a 8 (arriba, abajo, izquierda, derecha y las 4 diagonales).
    minas_colindantes = 0
    for fila in range(filas):
        for columna in range(columnas):
            if(fila > 0 and tableroBuscaminas[fila-1][columna] == mina): # La mina se encuentra arriba
                minas_colindantes += 1
            if(fila +1 < len(tableroBuscaminas) and tableroBuscaminas[fila+1][columna] == mina): # La mina se encuentra abajo
                minas_colindantes +=1
            if(columna+1 < len(tableroBuscaminas[0]) and tableroBuscaminas[fila][columna+1] == mina):
                # La mina se encuentra a la derecha
                minas_colindantes +=1
            if(columna > 0 and tableroBuscaminas[fila][columna-1] == mina): # La mina se encuentra a la izquierda
                minas_colindantes +=1
            if(fila > 0 and columna+1 < len(tableroBuscaminas[0]) and tableroBuscaminas[fila-1][columna+1] == mina):
                # La mina se encuentra en la diagonal superior derecha
                minas_colindantes +=1
            if(fila+1 < len(tableroBuscaminas) and columna+1 < len(tableroBuscaminas[0]) and 
                   tableroBuscaminas[fila+1][columna+1] == mina): 
                # La mina se encuentra en la diagonal inferior derecha
                minas_colindantes +=1
            if(fila > 0 and columna > 0 and tableroBuscaminas[fila-1][columna-1] == mina):
                # La mina se encuentra en la diagonal superior izquierda
                minas_colindantes +=1
            if(fila+1 < len(tableroBuscaminas) and columna > 0 and tableroBuscaminas[fila+1][columna-1] == mina): 
                # La mina se encuentra en la diagonal inferior izquierda
                minas_colindantes +=1
            # Pasamos a guardar el índice de minas colindantes en la casilla del tablero donde no hay ninguna mina.
            if(minas_colindantes > 0 and tableroBuscaminas[fila][columna] != mina ):
                tableroBuscaminas[fila][columna] = str(minas_colindantes)
            minas_colindantes=0
            
    return tableroBuscaminas
    
    
    
    

In [4]:
# Prueba de creación del tablero con los parámetros indicados
tableroBuscaminas = tablero_buscaminas(filas,columnas,minas)
# Para decirle que es un array y que salg
print(numpy.array(tableroBuscaminas))

[['☠' '2' '2' '3' '☠']
 ['1' '2' '☠' '☠' '4']
 ['0' '1' '3' '☠' '☠']
 ['0' '0' '1' '4' '☠']
 ['0' '0' '0' '2' '☠']]


In [5]:
# Pasamos a ocultar las casillas del tablero, se determina que dichas casillas ocultas se muestran con el símbolo "O"
def ocultar_casillas_tablero(filas,columnas):
    tablero_buscaminas_oculto = [[casilla_sin_pulsar for columna in range(columnas)] for fila in range(filas)]
        
    return tablero_buscaminas_oculto

In [6]:
# Prueba de ocultación del tablero
tablero_buscaminas_oculto = ocultar_casillas_tablero(filas,columnas)
print(numpy.array(tablero_buscaminas_oculto))

[['□' '□' '□' '□' '□']
 ['□' '□' '□' '□' '□']
 ['□' '□' '□' '□' '□']
 ['□' '□' '□' '□' '□']
 ['□' '□' '□' '□' '□']]


In [7]:
# Vamos ahora a destapar las casillas del tablero. Al clicar sobre una casilla que no contiene una mina, se van a destapar un
# conjunto colindante de casillas que no tienen minas ni están proximas con otras, es decir, con índice = 0. Como máximo,
# el límite del destapado va a ser si la casilla que se destapa tiene al menos un vecino con mina.
def destapar_casillas_tablero(fila,columna, tablero_buscaminas_oculto, tableroBuscaminas):
    # Guardo el valor de la casilla en la que hacemos click
    casillaDestapada = tableroBuscaminas[fila][columna]
    # Guardamos en una variable la longitud del tablero para optimizar código
    longitudTablero = len(tablero_buscaminas_oculto)

    # Si la casilla pulsada es una mina:
    if(casillaDestapada == mina):
        print(numpy.array(tableroBuscaminas))
        raise Exception("Has encontrado una mina. Has perdido")
    # Si no lo es, cambiamos su valor en el tablero oculto por el valor de la casilla destapada
    else:
        tablero_buscaminas_oculto[fila][columna] = casillaDestapada

    if(casillaDestapada == 0): #Comprobamos si la casilla que destapamos tiene 0 minas colindantes
        # si se da el caso, destapamos todas las casillas colindantes a la seleccionada ya que sabemos que ninguna es una mina
        # se comprueba que las colindantes estén dentro del tablero
        if((fila-1)>=0):
            destapar_casillas_tablero(fila-1,columna, tablero_buscaminas_oculto, tableroBuscaminas)
        if((fila+1)< longitudTablero):
            destapar_casillas_tablero(fila+1,columna, tablero_buscaminas_oculto, tableroBuscaminas)
        if((columna+1)< longitudTablero):
            destapar_casillas_tablero(fila,columna+1, tablero_buscaminas_oculto, tableroBuscaminas)
        if((columna-1)>=0):
            destapar_casillas_tablero(fila,columna-1, tablero_buscaminas_oculto, tableroBuscaminas)
        if((fila-1)>=0 and (columna+1) < longitudTablero):
            destapar_casillas_tablero(fila-1,columna+1, tablero_buscaminas_oculto, tableroBuscaminas)
        if((fila+1)< longitudTablero and (columna+1)< longitudTablero):
            destapar_casillas_tablero(fila+1,columna+1, tablero_buscaminas_oculto, tableroBuscaminas)
        if((fila-1)>=0 and (columna-1)>= 0):
            destapar_casillas_tablero(fila-1,columna-1, tablero_buscaminas_oculto, tableroBuscaminas)
        if((fila+1)< longitudTablero and (columna-1)>=0):
            destapar_casillas_tablero(fila+1,columna-1, tablero_buscaminas_oculto, tableroBuscaminas)

    return tablero_buscaminas_oculto


In [9]:
# Vamos a probar ahora el destapado de una casilla
destapar_casillas_tablero(0,3, tablero_buscaminas_oculto, tableroBuscaminas)

[['□', '□', '□', '3', '□'],
 ['□', '□', '□', '□', '□'],
 ['□', '□', '□', '□', '□'],
 ['□', '□', '□', '□', '□'],
 ['□', '□', '□', '□', '□']]

In [10]:
# Creamos un método para que se muestre los índices de las casillas vecinas a la destapada anteriormente. 
# Recorre las casillas colindantes a la casillas destapada y devuelve un array con la posición donde busca los indices.


def posicion_casillas_vecinas(fila,columna,tableroBuscaminas):

    indicesCasillas = []
    if((fila-1)>=0 and columna>=0):
        indicesCasillas.append("{0}{1}".format(fila-1, columna))
    if((fila+1)< len(tableroBuscaminas) and columna>=0):
        indicesCasillas.append("{0}{1}".format(fila+1, columna))
    if((fila>=0 and (columna+1) < len(tableroBuscaminas))):
        indicesCasillas.append("{0}{1}".format(fila, columna+1))
    if(fila>=0 and (columna-1)>=0):
        indicesCasillas.append("{0}{1}".format(fila, columna-1))
    if(((fila-1)>=0 and (columna+1) < len(tableroBuscaminas))):
        indicesCasillas.append("{0}{1}".format(fila-1, columna+1))
    if((fila+1)< len(tableroBuscaminas) and (columna+1)< len(tableroBuscaminas)):
        indicesCasillas.append("{0}{1}".format(fila+1, columna+1))
    if((fila-1)>=0 and (columna-1)>=0):
        indicesCasillas.append("{0}{1}".format(fila-1, columna-1))
    if((fila+1)< len(tableroBuscaminas) and (columna-1)>=0):
        indicesCasillas.append("{0}{1}".format(fila+1, columna-1))

    return indicesCasillas

In [11]:
# Ejemplo en relación con el anterior en el que se detapa la casilla. Puede servirnos en un futuro
posicion_casillas_vecinas(0,1,tableroBuscaminas)

['11', '02', '00', '12', '10']

In [12]:
def listado_de_minas(tableroBuscaminas):
    listado_minas = [] # Vamos a obtener las casillas donde están las minas situadas
    for fila in range(filas):
        for columna in range(columnas):
            if(tableroBuscaminas[fila][columna] == mina):
                listado_minas.append('({0},{1})'.format(fila,columna))
    return listado_minas


In [13]:
listado_de_minas(tableroBuscaminas)

['(0,0)', '(0,4)', '(1,2)', '(1,3)', '(2,3)', '(2,4)', '(3,4)', '(4,4)']

In [14]:
Minas_encontradas = []

In [44]:
# Pasamos a crear la red bayesiana
def red_bayesiana(filas,columnas,tablero_buscaminas_oculto):
    # Creating Bayesian Model with pgmpy library
    Buscaminas_bayesiano = pgmm.BayesianModel()

    # Creating the involved variables : Xij, Yij
    # Xij for squares with '?' (not revealed)
    # Yij for revealed squares that are not mine
    x_totales = []
    x_ocultas = []
    y_destapadas = []
    casilla_oculta_sin_colindantes_destapados = False
    for fila in range(filas):
        for columna in range(columnas):
            if (tablero_buscaminas_oculto[fila][columna] == casilla_sin_pulsar):
                x = 'Tapada{0}{1}'.format(fila,columna)

                x_totales.append(x)

                casilla_tiene_vecinos_destapados = False
                casillas_vecinas = posicion_casillas_vecinas(fila,columna,tableroBuscaminas) #obtiene la posicion de las casillas vecinas
                for vecino in casillas_vecinas:
                    if (tablero_buscaminas_oculto[int(vecino[0])][int(vecino[1])] != casilla_sin_pulsar): # comprueba que los vecinos de la casilla estan destapados
                        casilla_tiene_vecinos_destapados = True
                        break

                if (casilla_tiene_vecinos_destapados or not casilla_oculta_sin_colindantes_destapados):
                    x_ocultas.append(x)
                    Buscaminas_bayesiano.add_nodes_from([x])

                if (not casilla_tiene_vecinos_destapados):
                    casilla_oculta_sin_colidantes_destapados = True
            else:
                y = 'Destapada{0}{1}'.format(fila,columna)
                # y_destapadas.append(y)
                casilla_tiene_vecinos_ocultos = False
                casillas_vecinas = posicion_casillas_vecinas(fila,columna,tableroBuscaminas)
                Listado_de_minas = listado_de_minas(tableroBuscaminas)
                for vecino in casillas_vecinas:
                    casilla_vecina_oculta = 'Tapada{0}{1}'.format(vecino[0], vecino[1])
                    if (tablero_buscaminas_oculto[int(vecino[0])][int(vecino[1])] == casilla_sin_pulsar and
                        not Minas_encontradas.__contains__(casilla_vecina_oculta)):
                        casilla_tiene_vecinos_ocultos = True
                        break

                if (casilla_tiene_vecinos_ocultos):
                    y_destapadas.append(y)
                    Buscaminas_bayesiano.add_nodes_from([y])

    print("\nList of Xij nodes:\n")
    print(x_ocultas)
    print("\nList of Yij nodes:\n")
    print(y_destapadas)

    print("\nList of Xij nodes that are known to be mine:\n")
    print(Minas_encontradas)

    # Calculting Yij parent lists
    #	In order to get the CPDS of Yij squares, we will need their parent squares
    # Adding edges of Bayesian Model between nodes
    #	Yij nodes will have edges with all of its Xij parents
    Aristas  = dict()
    ListaColindantesOcultos = []
    for fila in range(filas):
        for columna in range(columnas):
            Destapada = 'Destapada{0}{1}'.format(fila,columna)
            if ((tablero_buscaminas_oculto[fila][columna] != casilla_sin_pulsar and y_destapadas.__contains__(Destapada))):
                casillas_vecinas = posicion_casillas_vecinas(fila,columna,tableroBuscaminas)
                for vecino in casillas_vecinas:
                    Tapada = 'Tapada{0}{1}'.format(vecino[0], vecino[1])
                    if (x_ocultas.__contains__(Tapada)):
                        Buscaminas_bayesiano.add_edges_from([(Tapada, Destapada)])
                        ListaColindantesOcultos.append(Tapada)

                Aristas.update({Destapada: ListaColindantesOcultos})
    
    print("Lista de aristas:\n")
    print(Aristas)
    print("\n")
    print("Lista de vecinos ocultos:\n")
    print(ListaColindantesOcultos)
    print("\n")

    print("Comprobamos la lista de aristas añadidas a la red bayesiana:\n") #Formato: [Tapada(i,j), Destapada(fijo,fijo)]
    print(Buscaminas_bayesiano.edges())
    print("\n")
    print("LISTA DE CASILLAS OCULTAS: \n")
    print(str(x_ocultas))

    # Creating the probability tables (CPDS)
    #	Xij CPDS will not depend of any other node
    #	Yij CPDS will depend of their parents Xij nodes
    ListaValoresDestapados = dict()
    for fila in range(filas):
        for columna in range(columnas):
            # X node
            Tapada = 'Tapada{0}{1}'.format(fila,columna)
            Destapada = 'Destapada{0}{1}'.format(fila,columna)
            if (tablero_buscaminas_oculto[fila][columna]==casilla_sin_pulsar and x_ocultas.__contains__(Tapada)):
                # Known mine
                if (Minas_encontradas.__contains__(Tapada)):
                    probabilidad_mina = 1
                # Probability to be mine
                elif (len(x_ocultas) > 0):
                    probabilidad_mina = (minas-len(Minas_encontradas))/len(x_totales)
                # There are no mines left, it cannot be mine
                else:
                    probabilidad_mina = 0

                # Adding CPD to bayesian model
                cpd = pgmf.TabularCPD(Tapada, 2, [[1-probabilidad_mina, probabilidad_mina]])
                Buscaminas_bayesiano.add_cpds(cpd)

            # Y node
            elif (tablero_buscaminas_oculto[fila][columna] != casilla_sin_pulsar and y_destapadas.__contains__(Destapada)):
                # Creating CPD table
                ListaValoresDestapados.update({Destapada: int(tablero_buscaminas_oculto[fila][columna])})
                nColumnasCPD  = pow(2, len(Aristas[Destapada]))

                numero_maximo_minas = 9

                # Square at the top or bottom of the board
                if (fila == 0 or fila == filas-1):
                    numero_maximo_minas = 6
                if (columna == 0 or columna == columnas-1):
                    # Corner square
                    if (numero_maximo_minas == 6):
                        numero_maximo_minas = 4
                    # Square at the left or right of the board
                    else:
                        numero_maximo_minas = 6

                if (len(Aristas[Destapada])+1 < numero_maximo_minas):
                    numero_maximo_minas = len(Aristas[Aristas])+1

                # Generating CPD table
                listaProb = []
                for numero_minas in range(numero_maximo_minas):
                    listaFila  = []
                    contador  = 0
                    for x in range(nColumnasCPD):
                        if (numero_minas == bin (contador).count('1')):
                            listaFila.append(1)
                        else:
                            listaFila.append(0)
                        contador += 1

                    listaProb.append(listaFila)

                ListaCardinalidad = []
                for _ in range(len(Aristas[Destapada])):
                    ListaCardinalidad.append(2)

                # Adding CPD to bayesian model
                cpd = pgmf.TabularCPD(Destapada, numero_maximo_minas, listaProb, Aristas[Destapada], ListaCardinalidad)
                Buscaminas_bayesiano.add_cpds(cpd)
                
                print("\nIMPRIMO TODOS LOS FACTORES DE LA CPD: \n")
                for m in Buscaminas_bayesiano.cpds: print(m)

    # Getting best Xij
    # Execute variable elimination method to get the queries
    Buscaminas_bayesiano_ev = pgmi.VariableElimination(Buscaminas_bayesiano)

    # Asking queries for all Xij nodes that are known to not be mine
    consultas = []
    for variable in x_ocultas:
        if(not variable in Minas_encontradas):
            consulta = Buscaminas_bayesiano_ev.query([variable], ListaValoresDestapados)
            consultas.append(consulta)


    # Creating dictionary with all the Xij queried and their probability to not be mine
    ocultasProbabilidadDeNoSerMina  = dict()
    for variable in consultas:
        x = ''
        probabilidadDeNoSerMina  = 0
        for var1 in variable.keys():
            x = str(var1)
        for var2 in variable.values():
            probabilidadDeNoSerMina  = float(var2.values[0])

        if (probabilidadDeNoSerMina  == 0.0):
            Listado_de_minas.append(x)

        ocultasProbabilidadDeNoSerMina.update({x: probabilidadDeNoSerMina })

    # Getting the best Xij with more probability to not be mine
    maximaProbabilidadDeNoSerMina  = 0.
    ocultaMaximaProbabilidadDeNoSerMina  = ''
    mejor_oculta = []
    print("\nGetting the best Xij with more probability to not be mine")
    for x, probabilidadDeNoSerMina in ocultasProbabilidadDeNoSerMina.items():
        print("x: {0} -> Probability to not be mine: {1}".format(x, probabilidadDeNoSerMina))
        if (maximaProbabilidadDeNoSerMina  < probabilidadDeNoSerMina):
            ocultaMaximaProbabilidadDeNoSerMina  = str(x)
            maximaProbabilidadDeNoSerMina  = probabilidadDeNoSerMina
            mejor_oculta = []
            mejor_oculta.append(ocultaMaximaProbabilidadDeNoSerMina )

        if (probabilidadDeNoSerMina == 1.0 and not x in mejor_oculta):
            mejor_oculta.append(x)

    # Refreshing list of best_x to not be mine
    mejor_oculta = mejor_oculta
    print("\nList of best Xij to not be mine: {0}".format(mejor_oculta))
    print(mejor_oculta)

    return Buscaminas_bayesiano
    


In [45]:
print(red_bayesiana(filas, columnas, tablero_buscaminas_oculto))


List of Xij nodes:

['Tapada00', 'Tapada01', 'Tapada02', 'Tapada04', 'Tapada10', 'Tapada11', 'Tapada12', 'Tapada13', 'Tapada14', 'Tapada20', 'Tapada21', 'Tapada22', 'Tapada23', 'Tapada24', 'Tapada30', 'Tapada31', 'Tapada32', 'Tapada33', 'Tapada34', 'Tapada40', 'Tapada41', 'Tapada42', 'Tapada43', 'Tapada44']

List of Yij nodes:

['Destapada03']

List of Xij nodes that are known to be mine:

['(0,0)', '(0,4)', '(1,2)', '(1,3)', '(2,3)', '(2,4)', '(3,4)', '(4,4)']
Lista de aristas:

{'Destapada03': ['Tapada13', 'Tapada04', 'Tapada02', 'Tapada14', 'Tapada12']}


Lista de vecinos ocultos:

['Tapada13', 'Tapada04', 'Tapada02', 'Tapada14', 'Tapada12']


Comprobamos la lista de aristas añadidas a la red bayesiana:

[('Tapada02', 'Destapada03'), ('Tapada04', 'Destapada03'), ('Tapada12', 'Destapada03'), ('Tapada13', 'Destapada03'), ('Tapada14', 'Destapada03')]


LISTA DE CASILLAS OCULTAS: 

['Tapada00', 'Tapada01', 'Tapada02', 'Tapada04', 'Tapada10', 'Tapada11', 'Tapada12', 'Tapada13', 'Tapada1


Getting the best Xij with more probability to not be mine
x: Tapada00 -> Probability to not be mine: 0.9166666666666666
x: Tapada01 -> Probability to not be mine: 0.9166666666666666
x: Tapada02 -> Probability to not be mine: 0.4
x: Tapada04 -> Probability to not be mine: 0.4
x: Tapada10 -> Probability to not be mine: 0.9166666666666666
x: Tapada11 -> Probability to not be mine: 0.9166666666666666
x: Tapada12 -> Probability to not be mine: 0.4
x: Tapada13 -> Probability to not be mine: 0.4
x: Tapada14 -> Probability to not be mine: 0.4
x: Tapada20 -> Probability to not be mine: 0.9166666666666666
x: Tapada21 -> Probability to not be mine: 0.9166666666666666
x: Tapada22 -> Probability to not be mine: 0.9166666666666666
x: Tapada23 -> Probability to not be mine: 0.9166666666666666
x: Tapada24 -> Probability to not be mine: 0.9166666666666666
x: Tapada30 -> Probability to not be mine: 0.9166666666666666
x: Tapada31 -> Probability to not be mine: 0.9166666666666666
x: Tapada32 -> Probabili